In [1]:
# import findspark
# findspark.init("/usr/local/spark")

In [2]:
from pyspark import SparkContext, SparkConf

In [3]:
import random
import subprocess
import os

In [4]:
sc = SparkContext()
def inside(p):
    x,y = random.random(), random.random()
    return x*x + y*y < 1
NUM_SAMPLES = 1000000
count = sc.parallelize(range(0, NUM_SAMPLES)).filter(inside).count()
print("Pi is roughtly %f"%(4.0 * count / NUM_SAMPLES))
sc.stop()

Pi is roughtly 3.138912


In [5]:
# start a spark task
sc = SparkContext()

要加载本地文件，必须采用 "file:///” 开头的这种格式。执行上上面这条命令以后，并不会马上显示结果，因为，Spark采用惰性机制，只有遇到“行动”类型的操作，才会从头到尾执行所有操作

In [6]:
filepath = "word.txt"
filepath = os.path.realpath(filepath)
print(filepath)
output = "output"
output = os.path.realpath(output)
print(output)

/media/fxy/study/GitHub/Job/Spark/code/test_spark/word.txt
/media/fxy/study/GitHub/Job/Spark/code/test_spark/output


In [38]:
#load txt file from local disk
local_disk = r"file://"
path = local_disk + filepath
print(path)
textFile = sc.textFile(path)

file:///media/fxy/study/GitHub/Job/Spark/code/test_spark/word.txt


first()是一个“行动”（Action）类型的操作，会启动真正的计算过程，从文件中加载数据到变量textFile中，并取出第一行文本。

In [35]:
textFile.first()

'Chinese researchers have developed interfacially polymerized porous polymer particles for low- abundance glycopeptide separation. These polymer particles – with hydrophilic-hydrophobic heterostructured nanopores – can separate low-abundance glycopeptides from complex biological samples with high-abundance background molecules efficiently. '

saveAsTextFile()是一个“行动”（Action）类型的操作，所以，马上会执行真正的计算过程，从word.txt中加载数据到变量textFile中，然后，又把textFile中的数据写回到本地文件目录

In [36]:
if os.path.exists(output) == False:
    out_path = local_disk + output
    textFile.saveAsTextFile(out_path)
else:
    print("folder exists!!!")

folder exists!!!


In [37]:
os.listdir(output)

['.part-00000.crc',
 '.part-00001.crc',
 '._SUCCESS.crc',
 'part-00000',
 'part-00001',
 '_SUCCESS']

### 加载HDFS文件 (只有hadoop用户才有权限)
主程序 hdfs_head = r"usr/local/hadoop/bin/hdfs
1. 创建一个hdfs 文件 /user/hadoop 这个目录是在HDFS文件系统中，不在本地文件系统中。
2. 讲本地文件导入hdfs文件系统中
3. 用pyspark 加载 hdfs中的文件

In [92]:
def do_cmd(cmd, print_cmd = False, input_pwd=False):
    if(print_cmd):
        print("cmd : ",cmd)
    p = subprocess.Popen(cmd, shell=True, 
                         stdin = subprocess.PIPE, stderr=subprocess.PIPE, 
                         stdout=subprocess.PIPE, universal_newlines=True)
        
    if(input_pwd):
        time.sleep(1)
        p.stdin.write("123\n")
    p.stdin.close()
    
    stderr = p.stderr
    if(stderr):
        print(stderr.readlines())
    if(not input_pwd):
        out = p.stdout.readlines()
        for row in out:
            print(row.strip())
    

In [53]:
def hdfs_ops(cmd=r"ls /", print_cmd=True):
    hdfs_head = r"/usr/local/hadoop/bin/hdfs dfs -"
    cmd = hdfs_head + cmd

    do_cmd(cmd, print_cmd)

In [60]:
# create folder spark
#hdfs_ops("rm -r /user")
hdfs_ops("mkdir -p /user/spark")
hdfs_ops("ls -R /")

cmd :  /usr/local/hadoop/bin/hdfs dfs -mkdir -p /user/spark
[]
cmd :  /usr/local/hadoop/bin/hdfs dfs -ls -R /
[]
drwxr-xr-x   - hadoop supergroup          0 2018-08-16 20:26 /user
drwxr-xr-x   - hadoop supergroup          0 2018-08-16 20:26 /user/spark


In [26]:
hdfs_ops("rm /user/spark/word.txt")

cmd :  /usr/local/hadoop/bin/hdfs dfs -rm /user/spark/word.txt


In [98]:
import time
def copy_to_hadoop(path, new_path = None, print_cmd = True):
    
    if(new_path is None):
        new_path = "/home/hadoop/mydata"
        
    print("\n-------copy from %s to %s----------"%(path, new_path))
    
    filename = os.path.split(path)[-1]
    new_path = os.path.join(new_path, filename)
    if(os.path.exists(new_path) == False):
        cmd1 = "sudo -S cp %s %s\n"%(path, new_path)
        do_cmd(cmd1,print_cmd, input_pwd=True)
    cmd2 = "sudo -S chown hadoop:hadoop %s"%new_path
    do_cmd(cmd2,print_cmd, input_pwd=True)
    cmd3 = "sudo -S chmod 664 %s"%new_path
    do_cmd(cmd3, print_cmd, input_pwd=True)
    cmd4 = "ls -l %s"%new_path
    do_cmd(cmd4, print_cmd)
    print("------------copy finished-----------\n")
    return new_path

In [100]:
print(filepath)
print(os.path.exists(filepath))
#header = r"file://"
#filepath2 = header + filepath
new_path = copy_to_hadoop(filepath)
hdfs_ops("put %s /user/spark/"%new_path)
hdfs_ops("ls /user/spark")

/media/fxy/study/GitHub/Job/Spark/code/test_spark/word.txt
True

-------copy from /media/fxy/study/GitHub/Job/Spark/code/test_spark/word.txt to None----------
cmd :  sudo -S chown hadoop:hadoop /home/hadoop/mydata/word.txt
['[sudo] fxy 的密码： ']
cmd :  sudo -S chmod 664 /home/hadoop/mydata/word.txt
['[sudo] fxy 的密码： ']
cmd :  ls -l /home/hadoop/mydata/word.txt
[]
-rw-rw-r-- 1 hadoop hadoop 2503 8月  16 22:09 /home/hadoop/mydata/word.txt
------------copy finished-----------

cmd :  /usr/local/hadoop/bin/hdfs dfs -put /home/hadoop/mydata/word.txt /user/spark/
["put: `/user/spark/word.txt': File exists\n"]
cmd :  /usr/local/hadoop/bin/hdfs dfs -ls /user/spark
[]
Found 1 items
-rw-r--r--   1 hadoop supergroup       2503 2018-08-16 22:31 /user/spark/word.txt


sc.textFile(“hdfs://localhost:9000/user/hadoop/word.txt”)中，“hdfs://localhost:9000/”是前面介绍Hadoop安装内容时确定下来的端口地址9000。实际上，也可以省略不写，如下三条语句都是等价的：
1. > textFile = sc.textFile("hdfs://localhost:9000/user/hadoop/word.txt")
2. > textFile = sc.textFile("/user/hadoop/word.txt")
3. > textFile = sc.textFile("word.txt")

In [115]:
textFile.first()

'Chinese researchers have developed interfacially polymerized porous polymer particles for low- abundance glycopeptide separation. These polymer particles – with hydrophilic-hydrophobic heterostructured nanopores – can separate low-abundance glycopeptides from complex biological samples with high-abundance background molecules efficiently. '

下面，我们再把textFile的内容写回到HDFS文件系统中（写到hadoop用户目录下）：

In [117]:
import shutil
output = "writeback"
if(os.path.exists(output)):
    shutil.rmtree(output)
textFile.saveAsTextFile(output)
print(os.listdir(output))

['.part-00000.crc', '.part-00001.crc', '._SUCCESS.crc', 'part-00000', 'part-00001', '_SUCCESS']


## 有了前面的铺垫性介绍，下面我们就可以开始第一个Spark应用程序：WordCount。


### 1. textFile.flatMap(labmda line : line.split(” “))
会遍历textFile中的每行文本内容，当遍历到其中一行文本内容时，会把文本内容赋值给变量line，并执行Lamda表达式line : line.split(” “).多行文本，采用空格作为分隔符进行单词切分，就得到多个单词集合

In [137]:
words = textFile.flatMap(lambda line: line.split(" "))

In [143]:
output = "words"
output = os.path.realpath(output)
if(os.path.exists(output)):
    shutil.rmtree(output)
    
words.saveAsTextFile(output)

tmp = os.path.join(output,"part-00000")
with open(tmp,"r") as f:
    print(f.readlines())

['Chinese\n', 'researchers\n', 'have\n', 'developed\n', 'interfacially\n', 'polymerized\n', 'porous\n', 'polymer\n', 'particles\n', 'for\n', 'low-\n', 'abundance\n', 'glycopeptide\n', 'separation.\n', 'These\n', 'polymer\n', 'particles\n', '–\n', 'with\n', 'hydrophilic-hydrophobic\n', 'heterostructured\n', 'nanopores\n', '–\n', 'can\n', 'separate\n', 'low-abundance\n', 'glycopeptides\n', 'from\n', 'complex\n', 'biological\n', 'samples\n', 'with\n', 'high-abundance\n', 'background\n', 'molecules\n', 'efficiently.\n', '\n', '\n', 'Results\n', 'were\n', 'published\n', 'in\n', 'the\n', 'journal\n', 'Advanced\n', 'Materials\n', 'in\n', 'an\n', 'article\n', 'entitled\n', '"Interfacially\n', 'Polymerized\n', 'Particles\n', 'with\n', 'Heterostructured\n', 'Nanopores\n', 'for\n', 'Glycopeptide\n', 'Separation."\n', '\n', '\n', 'Qualitative/quantitative\n', 'analyses\n', 'of\n', 'low-abundance\n', 'biomolecules\n', 'from\n', 'complex\n', 'biofluids\n', 'are\n', 'critical\n', 'in\n', 'clinical\n'

### 2. word : (word, 1)
这个Lamda表达式的含义是，word作为函数的输入参数，然后，执行函数处理逻辑，这里会执行(word, 1)，也就是针对输入的word，构建得到一个tuple，形式为(word,1)，key是word，value是1（表示该单词出现1次）

In [144]:
word_tuples = words.map(lambda word:(word,1))

In [145]:
output = "word_tuples"
output = os.path.realpath(output)
if(os.path.exists(output)):
    shutil.rmtree(output)
    
word_tuples.saveAsTextFile(output)

tmp = os.path.join(output,"part-00000")
with open(tmp,"r") as f:
    print(f.readlines())

["('Chinese', 1)\n", "('researchers', 1)\n", "('have', 1)\n", "('developed', 1)\n", "('interfacially', 1)\n", "('polymerized', 1)\n", "('porous', 1)\n", "('polymer', 1)\n", "('particles', 1)\n", "('for', 1)\n", "('low-', 1)\n", "('abundance', 1)\n", "('glycopeptide', 1)\n", "('separation.', 1)\n", "('These', 1)\n", "('polymer', 1)\n", "('particles', 1)\n", "('–', 1)\n", "('with', 1)\n", "('hydrophilic-hydrophobic', 1)\n", "('heterostructured', 1)\n", "('nanopores', 1)\n", "('–', 1)\n", "('can', 1)\n", "('separate', 1)\n", "('low-abundance', 1)\n", "('glycopeptides', 1)\n", "('from', 1)\n", "('complex', 1)\n", "('biological', 1)\n", "('samples', 1)\n", "('with', 1)\n", "('high-abundance', 1)\n", "('background', 1)\n", "('molecules', 1)\n", "('efficiently.', 1)\n", "('', 1)\n", "('', 1)\n", "('Results', 1)\n", "('were', 1)\n", "('published', 1)\n", "('in', 1)\n", "('the', 1)\n", "('journal', 1)\n", "('Advanced', 1)\n", "('Materials', 1)\n", "('in', 1)\n", "('an', 1)\n", "('article', 1)\n

### reduceByKey(labmda a, b : a + b)操作，
得到一个RDD，这个RDD的每个元素是(key,value)形式的tuple。最后，针对这个RDD，执行reduceByKey(labmda a, b : a + b)操作，这个操作会把所有RDD元素按照key进行分组，然后使用给定的函数（这里就是Lamda表达式：a, b : a + b），对具有相同的key的多个value进行reduce操作，返回reduce后的(key,value)，比如(“hadoop”,1)和(“hadoop”,1)，具有相同的key，进行reduce以后就得到(“hadoop”,2)，

In [146]:
wordCount = word_tuples.reduceByKey(lambda a, b : a+b)

In [161]:
output = "wordCount"
output = os.path.realpath(output)
if(os.path.exists(output)):
    shutil.rmtree(output)
    
wordCount.saveAsTextFile(output)

tmp = os.path.join(output,"part-00000")
with open(tmp,"r") as f:
    cnt = 1
    for row in f.readlines():
        print("%30s"%row.strip(), end=' ')
        if cnt == 3:
            cnt = 0
            print('')
        cnt += 1

            ('researchers', 2)                    ('have', 4)               ('developed', 2) 
          ('interfacially', 1)             ('polymerized', 1)               ('particles', 4) 
              ('abundance', 3)             ('separation.', 2)                   ('These', 1) 
          ('low-abundance', 4)           ('glycopeptides', 4)               ('molecules', 3) 
           ('efficiently.', 1)                       ('', 16)               ('published', 1) 
                     ('in', 5)                      ('an', 2)                ('entitled', 1) 
            ('Polymerized', 1)               ('Nanopores', 1)            ('Glycopeptide', 1) 
           ('Separation."', 1)                ('analyses', 1)                     ('of', 12) 
           ('biomolecules', 3)               ('biofluids', 1)                     ('are', 2) 
              ('diagnosis', 1)                ('example,', 1)                 ('peptide', 1) 
                   ('help', 2)                ('identify', 2

In [152]:
#the output in the terminal, sad
#wordCount.foreachPartition(myprint)